In [1]:
## importing
import pandas as pd
import numpy as np
import sys
import os
import math
from scipy.stats import beta, uniform, norm
from statsmodels.stats.multitest import multipletests
from collections import OrderedDict
import subprocess
## loading hugo names
hugo = pd.read_csv('/Users/leyang/Dropbox/Projects/Subnetwork_survey/code/Database/hugo_symbol.txt')
hugo = list(hugo['Approved symbol'])

In [2]:
## Generate gene list etc from p-values, locfdr

#load genes
tmp = pd.read_csv('./networks/merge_bioirefreastr_index_gene',sep='\t',header=None)
total_gene = list(tmp[1])
tmp_bio = pd.read_csv('./networks/biogrid_index_gene',sep='\t',header=None)
bio_gene = list(tmp_bio[1])
tmp_iref = pd.read_csv('./networks/irefindex18_index_gene',sep='\t',header=None)
iref_gene = list(tmp_iref[1])
tmp_rea = pd.read_csv('./networks/reactome21_index_gene',sep='\t',header=None)
rea_gene = list(tmp_rea[1])
tmp_str = pd.read_csv('./networks/string_index_gene',sep='\t',header=None)
str_gene = list(tmp_str[1])

base_path = '/Users/leyang/Dropbox/Projects/Subnetwork_survey'

data_type = "real"
data_list = ["BLCA_MERGE_v1","LUAD_MERGE_v1","BLCA_MERGE_v2","LUAD_MERGE_v2","BLCA_MERGE_v3","LUAD_MERGE_v3","BLCA_MERGE_v4",
             "LUAD_MERGE_v4","BLCA_SNV","LUAD_SNV","BRCA_SNV","LUSC_MERGE_v1","COADREAD_MERGE_v1","LUSC_MERGE_v2","COADREAD_MERGE_v2",
             "LUSC_MERGE_v3","COADREAD_MERGE_v3","LUSC_MERGE_v4","COADREAD_MERGE_v4","LUSC_SNV","COADREAD_SNV","PRAD_MERGE_v1",
             "HNSC_MERGE_v1","PRAD_MERGE_v2","HNSC_MERGE_v2","PRAD_MERGE_v3","HNSC_MERGE_v3","PRAD_MERGE_v4","HNSC_MERGE_v4",
             "PRAD_SNV","HNSC_SNV","UCEC_MERGE_v1","KIPAN_MERGE_v1","UCEC_MERGE_v2","KIPAN_MERGE_v2","UCEC_MERGE_v3","KIPAN_MERGE_v3",
             "UCEC_MERGE_v4","KIPAN_MERGE_v4","UCEC_SNV","KIPAN_SNV","BRCA_MERGE_v4"]

path = os.path.join(base_path,'code','score',data_type)
path_p = os.path.join(base_path,'code','score',data_type,'p')
path_q = os.path.join(base_path,'code','score',data_type,'q')
path_z = os.path.join(base_path,'code','score',data_type,'z')
path_lfdr = os.path.join(base_path,'code','score',data_type,'lfdr')
path_list = os.path.join(base_path,'code','score',data_type,'list')
path_clustex = os.path.join(base_path,'code','score',data_type,'clustex')
path_diamond = os.path.join(base_path,'code','score',data_type,'diamond')
path_all = [path_p,path_q,path_z,path_lfdr,path_list,path_clustex,path_diamond]

if not os.path.exists(path):
    os.makedirs(path)
for pa in path_all:
    if not os.path.exists(pa):
        os.makedirs(pa)



In [3]:
len(data_list)

42

In [29]:
name = "BLCA_MERGE_v1"
tmp_p = pd.read_csv(os.path.join(path_p, data + '_p.txt'),sep='\t',header=None)
p_dict = dict(zip(tmp_p[0],tmp_p[1]))
p_vals = [p_dict[k] for k in p_dict.keys()]
# calculate z-values
z_vals = [-norm.ppf(p) for p in p_vals]
z_dict = dict(zip(p_dict.keys(),z_vals))

# calculate lfdr
tmp_lfdr = pd.read_csv(os.path.join(path_lfdr, data + '_fdr.txt'),sep='\t',header=None)
fdr_dict = dict(zip(tmp_lfdr[0],tmp_lfdr[1]))

fdr = [1.0 for i in range(len(z_vals))]
for i in range(len(z_vals)):
    if z_vals[i] == float('Inf'):
        fdr[i] = 0
        print("Inf")
for i in range(len(fdr)):
    if z_vals[i] < 0:
        fdr[i] = 1.0
    if (z_vals[i] > 0) and math.isnan(fdr[i]):
        fdr[i] = 0.0
lfdr_dict = dict(zip(p_dict.keys(),fdr))

# gene list
gene_list = [k for k,v in lfdr_dict.items() if v <= 0.1]
gene_list_index_bio = [bio_gene.index(g) + 1 for g in gene_list if g in bio_gene]
gene_list_index_iref = [iref_gene.index(g) + 1 for g in gene_list if g in iref_gene]
gene_list_index_rea = [rea_gene.index(g) + 1 for g in gene_list if g in rea_gene]
gene_list_index_str = [str_gene.index(g) + 1 for g in gene_list if g in str_gene]

Inf
Inf


In [33]:
tmp_pp_dict = dict(zip(tmp_p[0],tmp_p[1]))
print(tmp_pp_dict)

{'A1BG': 0.689714518696074, 'A1CF': 0.999996779440313, 'A2BP1': 0.99018856418908, 'A2M': 0.755015974555607, 'A2ML1': 0.901685367830651, 'A4GALT': 0.931942060105946, 'A4GNT': 0.998497179555996, 'AAAS': 0.994791118121911, 'AACS': 0.999999999916396, 'AADAC': 0.981179095104874, 'AADACL2': 0.995866440767999, 'AADACL3': 0.998476071224553, 'AADACL4': 0.980141369465035, 'AADAT': 0.649950824448289, 'AAGAB': 0.540072056095877, 'AAK1': 0.981236971968931, 'AAMP': 0.266456624500317, 'AANAT': 0.986707550658825, 'AARS': 0.39906077708352, 'AARS2': 0.102825281993357, 'AARSD1': 0.868256246508116, 'AASDH': 0.349254665829888, 'AASDHPPT': 0.287591937374178, 'AASS': 0.985214785040749, 'AATF': 0.19838081526642, 'AATK': 0.796434333903015, 'ABAT': 0.426169146421751, 'ABCA1': 0.794101193911939, 'ABCA10': 0.658412977879907, 'ABCA11P': 0.742221276661172, 'ABCA12': 0.966241307816872, 'ABCA13': 0.995246133214303, 'ABCA17P': 1.0, 'ABCA2': 0.263071970220996, 'ABCA3': 0.87231538757888, 'ABCA4': 0.999936663006146, 'ABC

In [5]:
for name in data_list:
    # load p-values
    tmp_p = pd.read_csv(os.path.join(path_p,name + '_p.txt'),sep='\t',header=None)
    p_dict = dict(zip(tmp_p[0],tmp_p[1]))
    # calculate z-values
    z_dict = {}
    for k in p_dict.keys():
        z_dict[k] = -norm.ppf(p_dict[k])
    # calcualte q-values
    p_vals = [p_dict[k] for k in p_dict.keys()]
    tmp,p_adjusted,tmp,tmp = multipletests(p_vals,method='fdr_bh')
    q_score = [-math.log10(p) if p!=0 else 200 for p in p_adjusted]
    q_dict = dict(zip(p_dict.keys(),q_score))
    
    # load lfdr
    tmp_lfdr = pd.read_csv(os.path.join(path_lfdr,name + '_fdr.txt'),sep='\t',header=None)
    lfdr_dict = dict(zip(tmp_lfdr[0],tmp_lfdr[1]))
    for k in lfdr_dict.keys():
        if z_dict[k] == float('Inf'):
            lfdr_dict[k] = 0
        if z_dict[k] < 0:
            lfdr_dict[k] = 1.0
        if (z_dict[k] > 0) and math.isnan(lfdr_dict[k]):
            lfdr_dict[k] = 0.0
    
    # gene list
    gene_list = [k for k,v in lfdr_dict.items() if v <= 0.1]
    gene_list_index_bio = [bio_gene.index(g) + 1 for g in gene_list if g in bio_gene]
    gene_list_index_iref = [iref_gene.index(g) + 1 for g in gene_list if g in iref_gene]
    gene_list_index_rea = [rea_gene.index(g) + 1 for g in gene_list if g in rea_gene]
    gene_list_index_str = [str_gene.index(g) + 1 for g in gene_list if g in str_gene]
    
    
    # save all scores to files
    q_df = pd.DataFrame(sorted(q_dict.items()),columns=['Gene','q'])
    q_df.to_csv(os.path.join(path_q,name+'_q.txt'),header=None,sep='\t',index=False)
    z_df = pd.DataFrame(sorted(z_dict.items()),columns=['Gene','z'])
    z_df.to_csv(os.path.join(path_z,name+'_z.txt'),header=None,sep='\t',index=False)
    
    # gene list
    with open(os.path.join(path_list,name+'_list.txt'),'w') as file:
        for g in gene_list:
            file.write("%s\n" % g)
            
    # clustex biogrid
    with open(os.path.join(path_clustex,name+'_clustex_biogrid.txt'),'w') as file:
        file.write("Gene\n")
        for g in gene_list_index_bio:
            file.write("%s\n" % g)
    # clustex irefindex
    with open(os.path.join(path_clustex,name+'_clustex_irefindex18.txt'),'w') as file:
        file.write("Gene\n")
        for g in gene_list_index_iref:
            file.write("%s\n" % g)
    # clustex reactome21
    with open(os.path.join(path_clustex,name+'_clustex_reactome21.txt'),'w') as file:
        file.write("Gene\n")
        for g in gene_list_index_rea:
            file.write("%s\n" % g)
    # clustex string
    with open(os.path.join(path_clustex,name+'_clustex_string.txt'),'w') as file:
        file.write("Gene\n")
        for g in gene_list_index_str:
            file.write("%s\n" % g)
            
    # diamond biogrid
    with open(os.path.join(path_diamond,name+'_diamond_biogrid.txt'),'w') as file:
        for g in gene_list_index_bio:
            file.write("%s\n" % g)
    # diamond irefindex
    with open(os.path.join(path_diamond,name+'_diamond_irefindex18.txt'),'w') as file:
        for g in gene_list_index_iref:
            file.write("%s\n" % g)
    # diamond reactome21
    with open(os.path.join(path_diamond,name+'_diamond_reactome21.txt'),'w') as file:
        for g in gene_list_index_rea:
            file.write("%s\n" % g)
    # diamond string
    with open(os.path.join(path_diamond,name+'_diamond_string.txt'),'w') as file:
        for g in gene_list_index_str:
            file.write("%s\n" % g)
                


In [7]:
## send networks and scores to CCR
data_version_list = ["corum_"+str(i) for i in range(5)] +  ["reactome_"+str(i) for i in range(5)]
beta = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11]
permu = [1,2,3,4,5,6,7,8,9,10]
network_list = ["biogrid","irefindex18","reactome21","string"]

## data
fdrdata = [os.path.join("./","score",data_version,"lfdr",data_version+"_lfdr_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list]
pdata = [os.path.join("./","score",data_version,"p",data_version+"_p_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list]
qdata = [os.path.join("./","score",data_version,"q",data_version+"_q_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list]
zdata = [os.path.join("./","score",data_version,"z",data_version+"_z_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list]
genelistdata = [os.path.join("./","score",data_version,"list",data_version+"_list_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list]
clustexdata = [os.path.join("./","score",data_version,"clustex",data_version+"_clustex_"+network+"_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list for network in network_list]
diamonddata = [os.path.join("./","score",data_version,"diamond",data_version+"_diamond_"+network+"_beta_"+str(b)+"_"+str(p)+".txt") for b in beta for p in permu for data_version in data_version_list for network in network_list]
## network
network_index_gene = [os.path.join("./","networks",network+"_index_gene") for network in network_list]
network_edge_list = [os.path.join("./","networks",network+"_edge_list") for network in network_list]
network_edge_sif = [os.path.join("./","networks",network+"_edge_list.sif") for network in network_list]
network_edge_tsv = [os.path.join("./","networks",network+"_edge_list.tsv") for network in network_list]
network_clustex = [os.path.join("./","networks",network+"_edge_list_clustex.txt") for network in network_list]
## ensure data exists
for f in fdrdata + pdata + qdata + zdata + genelistdata + clustexdata + diamonddata + network_index_gene + network_edge_list + network_edge_sif + network_edge_tsv:
    assert os.path.exists(f)

In [14]:
#send to domino
subprocess.run(["scp",*network_edge_sif,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/domino/network/"])
subprocess.run(["scp",*genelistdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/domino/data/"])


CompletedProcess(args=['scp', './score/corum_0/list/corum_0_list_beta_0.01_1.txt', './score/corum_1/list/corum_1_list_beta_0.01_1.txt', './score/corum_2/list/corum_2_list_beta_0.01_1.txt', './score/corum_3/list/corum_3_list_beta_0.01_1.txt', './score/corum_4/list/corum_4_list_beta_0.01_1.txt', './score/reactome_0/list/reactome_0_list_beta_0.01_1.txt', './score/reactome_1/list/reactome_1_list_beta_0.01_1.txt', './score/reactome_2/list/reactome_2_list_beta_0.01_1.txt', './score/reactome_3/list/reactome_3_list_beta_0.01_1.txt', './score/reactome_4/list/reactome_4_list_beta_0.01_1.txt', './score/corum_0/list/corum_0_list_beta_0.01_2.txt', './score/corum_1/list/corum_1_list_beta_0.01_2.txt', './score/corum_2/list/corum_2_list_beta_0.01_2.txt', './score/corum_3/list/corum_3_list_beta_0.01_2.txt', './score/corum_4/list/corum_4_list_beta_0.01_2.txt', './score/reactome_0/list/reactome_0_list_beta_0.01_2.txt', './score/reactome_1/list/reactome_1_list_beta_0.01_2.txt', './score/reactome_2/list/re

In [13]:
# send to diamond
subprocess.run(["scp",*network_edge_list,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/diamond/network/"])
subprocess.run(["scp",*diamonddata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/diamond/data/"])


CompletedProcess(args=['scp', './score/corum_0/diamond/corum_0_diamond_biogrid_beta_0.01_1.txt', './score/corum_0/diamond/corum_0_diamond_irefindex18_beta_0.01_1.txt', './score/corum_0/diamond/corum_0_diamond_reactome21_beta_0.01_1.txt', './score/corum_0/diamond/corum_0_diamond_string_beta_0.01_1.txt', './score/corum_1/diamond/corum_1_diamond_biogrid_beta_0.01_1.txt', './score/corum_1/diamond/corum_1_diamond_irefindex18_beta_0.01_1.txt', './score/corum_1/diamond/corum_1_diamond_reactome21_beta_0.01_1.txt', './score/corum_1/diamond/corum_1_diamond_string_beta_0.01_1.txt', './score/corum_2/diamond/corum_2_diamond_biogrid_beta_0.01_1.txt', './score/corum_2/diamond/corum_2_diamond_irefindex18_beta_0.01_1.txt', './score/corum_2/diamond/corum_2_diamond_reactome21_beta_0.01_1.txt', './score/corum_2/diamond/corum_2_diamond_string_beta_0.01_1.txt', './score/corum_3/diamond/corum_3_diamond_biogrid_beta_0.01_1.txt', './score/corum_3/diamond/corum_3_diamond_irefindex18_beta_0.01_1.txt', './score/c

In [6]:
# send to must
# network
subprocess.run(["scp",*network_edge_tsv,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/must/network/"])
subprocess.run(["scp",*genelistdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/must/data/"])


CompletedProcess(args=['scp', './score/corum_0/list/corum_0_list_beta_0.01_1.txt', './score/corum_1/list/corum_1_list_beta_0.01_1.txt', './score/corum_2/list/corum_2_list_beta_0.01_1.txt', './score/corum_3/list/corum_3_list_beta_0.01_1.txt', './score/corum_4/list/corum_4_list_beta_0.01_1.txt', './score/reactome_0/list/reactome_0_list_beta_0.01_1.txt', './score/reactome_1/list/reactome_1_list_beta_0.01_1.txt', './score/reactome_2/list/reactome_2_list_beta_0.01_1.txt', './score/reactome_3/list/reactome_3_list_beta_0.01_1.txt', './score/reactome_4/list/reactome_4_list_beta_0.01_1.txt', './score/corum_0/list/corum_0_list_beta_0.01_2.txt', './score/corum_1/list/corum_1_list_beta_0.01_2.txt', './score/corum_2/list/corum_2_list_beta_0.01_2.txt', './score/corum_3/list/corum_3_list_beta_0.01_2.txt', './score/corum_4/list/corum_4_list_beta_0.01_2.txt', './score/reactome_0/list/reactome_0_list_beta_0.01_2.txt', './score/reactome_1/list/reactome_1_list_beta_0.01_2.txt', './score/reactome_2/list/re

In [9]:
# send to clustex
# network
subprocess.run(["scp",*network_clustex,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/clustex/network/"])
subprocess.run(["scp",*clustexdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/clustex/data/"])


CompletedProcess(args=['scp', './score/corum_0/clustex/corum_0_clustex_biogrid_beta_0.01_1.txt', './score/corum_0/clustex/corum_0_clustex_irefindex18_beta_0.01_1.txt', './score/corum_0/clustex/corum_0_clustex_reactome21_beta_0.01_1.txt', './score/corum_0/clustex/corum_0_clustex_string_beta_0.01_1.txt', './score/corum_1/clustex/corum_1_clustex_biogrid_beta_0.01_1.txt', './score/corum_1/clustex/corum_1_clustex_irefindex18_beta_0.01_1.txt', './score/corum_1/clustex/corum_1_clustex_reactome21_beta_0.01_1.txt', './score/corum_1/clustex/corum_1_clustex_string_beta_0.01_1.txt', './score/corum_2/clustex/corum_2_clustex_biogrid_beta_0.01_1.txt', './score/corum_2/clustex/corum_2_clustex_irefindex18_beta_0.01_1.txt', './score/corum_2/clustex/corum_2_clustex_reactome21_beta_0.01_1.txt', './score/corum_2/clustex/corum_2_clustex_string_beta_0.01_1.txt', './score/corum_3/clustex/corum_3_clustex_biogrid_beta_0.01_1.txt', './score/corum_3/clustex/corum_3_clustex_irefindex18_beta_0.01_1.txt', './score/c

In [5]:
# send to netcore
# network
subprocess.run(["scp",*network_edge_tsv,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netcore/network/"])
subprocess.run(["scp",*genelistdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netcore/data/"])


CompletedProcess(args=['scp', './score/corum_0/list/corum_0_list_beta_0.01_1.txt', './score/corum_1/list/corum_1_list_beta_0.01_1.txt', './score/corum_2/list/corum_2_list_beta_0.01_1.txt', './score/corum_3/list/corum_3_list_beta_0.01_1.txt', './score/corum_4/list/corum_4_list_beta_0.01_1.txt', './score/reactome_0/list/reactome_0_list_beta_0.01_1.txt', './score/reactome_1/list/reactome_1_list_beta_0.01_1.txt', './score/reactome_2/list/reactome_2_list_beta_0.01_1.txt', './score/reactome_3/list/reactome_3_list_beta_0.01_1.txt', './score/reactome_4/list/reactome_4_list_beta_0.01_1.txt', './score/corum_0/list/corum_0_list_beta_0.01_2.txt', './score/corum_1/list/corum_1_list_beta_0.01_2.txt', './score/corum_2/list/corum_2_list_beta_0.01_2.txt', './score/corum_3/list/corum_3_list_beta_0.01_2.txt', './score/corum_4/list/corum_4_list_beta_0.01_2.txt', './score/reactome_0/list/reactome_0_list_beta_0.01_2.txt', './score/reactome_1/list/reactome_1_list_beta_0.01_2.txt', './score/reactome_2/list/re

In [4]:
# send to robust
# network
subprocess.run(["scp",*network_edge_tsv,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/robust/network/"])
subprocess.run(["scp",*genelistdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/robust/data/"])



CompletedProcess(args=['scp', './score/corum_0/list/corum_0_list_beta_0.01_1.txt', './score/corum_1/list/corum_1_list_beta_0.01_1.txt', './score/corum_2/list/corum_2_list_beta_0.01_1.txt', './score/corum_3/list/corum_3_list_beta_0.01_1.txt', './score/corum_4/list/corum_4_list_beta_0.01_1.txt', './score/reactome_0/list/reactome_0_list_beta_0.01_1.txt', './score/reactome_1/list/reactome_1_list_beta_0.01_1.txt', './score/reactome_2/list/reactome_2_list_beta_0.01_1.txt', './score/reactome_3/list/reactome_3_list_beta_0.01_1.txt', './score/reactome_4/list/reactome_4_list_beta_0.01_1.txt', './score/corum_0/list/corum_0_list_beta_0.01_2.txt', './score/corum_1/list/corum_1_list_beta_0.01_2.txt', './score/corum_2/list/corum_2_list_beta_0.01_2.txt', './score/corum_3/list/corum_3_list_beta_0.01_2.txt', './score/corum_4/list/corum_4_list_beta_0.01_2.txt', './score/reactome_0/list/reactome_0_list_beta_0.01_2.txt', './score/reactome_1/list/reactome_1_list_beta_0.01_2.txt', './score/reactome_2/list/re

In [11]:
# send to BioNet
# network
#subprocess.run(["scp",*network_index_gene,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/bionet/network/"])
#subprocess.run(["scp",*network_edge_list,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/bionet/network/"])
# score
subprocess.run(["scp",*pdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/bionet/data/"])
# send to hotnet2
subprocess.run(["scp",*qdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/hotnet2/survey/data/heats/"])
# send to netmix2
subprocess.run(["scp",*pdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netmix2/data/"])
#subprocess.run(["scp",*network_edge_tsv,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netmix2/network/"])
# send to FDRnet
subprocess.run(["scp",*fdrdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/fdrnet/data/"])
# network




CompletedProcess(args=['scp', './score/corum_0/lfdr/corum_0_lfdr_beta_0.01_1.txt', './score/corum_1/lfdr/corum_1_lfdr_beta_0.01_1.txt', './score/corum_2/lfdr/corum_2_lfdr_beta_0.01_1.txt', './score/corum_3/lfdr/corum_3_lfdr_beta_0.01_1.txt', './score/corum_4/lfdr/corum_4_lfdr_beta_0.01_1.txt', './score/reactome_0/lfdr/reactome_0_lfdr_beta_0.01_1.txt', './score/reactome_1/lfdr/reactome_1_lfdr_beta_0.01_1.txt', './score/reactome_2/lfdr/reactome_2_lfdr_beta_0.01_1.txt', './score/reactome_3/lfdr/reactome_3_lfdr_beta_0.01_1.txt', './score/reactome_4/lfdr/reactome_4_lfdr_beta_0.01_1.txt', './score/corum_0/lfdr/corum_0_lfdr_beta_0.01_2.txt', './score/corum_1/lfdr/corum_1_lfdr_beta_0.01_2.txt', './score/corum_2/lfdr/corum_2_lfdr_beta_0.01_2.txt', './score/corum_3/lfdr/corum_3_lfdr_beta_0.01_2.txt', './score/corum_4/lfdr/corum_4_lfdr_beta_0.01_2.txt', './score/reactome_0/lfdr/reactome_0_lfdr_beta_0.01_2.txt', './score/reactome_1/lfdr/reactome_1_lfdr_beta_0.01_2.txt', './score/reactome_2/lfdr/re

CompletedProcess(args=['scp', './score/corum_0/q/corum_0_q_beta_0.01_1.txt', './score/corum_1/q/corum_1_q_beta_0.01_1.txt', './score/corum_2/q/corum_2_q_beta_0.01_1.txt', './score/corum_3/q/corum_3_q_beta_0.01_1.txt', './score/corum_4/q/corum_4_q_beta_0.01_1.txt', './score/reactome_0/q/reactome_0_q_beta_0.01_1.txt', './score/reactome_1/q/reactome_1_q_beta_0.01_1.txt', './score/reactome_2/q/reactome_2_q_beta_0.01_1.txt', './score/reactome_3/q/reactome_3_q_beta_0.01_1.txt', './score/reactome_4/q/reactome_4_q_beta_0.01_1.txt', './score/corum_0/q/corum_0_q_beta_0.01_2.txt', './score/corum_1/q/corum_1_q_beta_0.01_2.txt', './score/corum_2/q/corum_2_q_beta_0.01_2.txt', './score/corum_3/q/corum_3_q_beta_0.01_2.txt', './score/corum_4/q/corum_4_q_beta_0.01_2.txt', './score/reactome_0/q/reactome_0_q_beta_0.01_2.txt', './score/reactome_1/q/reactome_1_q_beta_0.01_2.txt', './score/reactome_2/q/reactome_2_q_beta_0.01_2.txt', './score/reactome_3/q/reactome_3_q_beta_0.01_2.txt', './score/reactome_4/q/r

CompletedProcess(args=['scp', './networks/biogrid_edge_list.tsv', './networks/irefindex18_edge_list.tsv', './networks/reactome21_edge_list.tsv', './networks/string_edge_list.tsv', 'lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netmix2/network/'], returncode=0)

In [28]:

subprocess.run(["scp",*network_index_gene,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/fdrnet/network/"])
subprocess.run(["scp",*network_edge_list,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/fdrnet/network/"])

CompletedProcess(args=['scp', './networks/biogrid_edge_list', './networks/irefindex18_edge_list', './networks/reactome21_edge_list', './networks/string_edge_list', 'lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/fdrnet/network/'], returncode=0)

In [ ]:
# send to FDRnet
subprocess.run(["scp",*fdrdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/data/"])

# send to hhotnet
subprocess.run(["scp",*qdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/hhotnet/biogrid/Data/"])

# send to netmix2
subprocess.run(["scp",*pdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/netmix2/data/"])

# send to domino
subprocess.run(["scp",*genelistdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/domino/Data/"])

# send to BioNet
subprocess.run(["scp",*pdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/bionet/data/"])

# send to clustex
subprocess.run(["scp",*clustexdata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/clustex/data/"])

# send to diamond
subprocess.run(["scp",*diamonddata,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey/diamond/data/"])


In [ ]:
# # send to regmod
# subprocess.run(["scp",*data_mat,"lyang25@vortex.ccr.buffalo.edu:/projects/academic/yijunsun/leyang/survey_others/regmod/data/"])


In [12]:
data_path = "./data";
network_path = "./network"
result_folder = "./result"
network_list = ["biogrid","irefindex18","reactome21","string"]
network = network_list*1100#[net for net in network_list for i in range(1100)]
data_version_list = ["corum_"+str(k) for k in range(5)] + ["reactome_"+str(k) for k in range(5)]
beta = [0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,0.1,0.11]
permu = [1,2,3,4,5,6,7,8,9,10]
score = ["".join((data_version,"_diamond_"+net+"_beta_",str(b),"_",str(p))) for b in beta for p in permu for data_version in data_version_list for net in network_list]
#python run_netmix2.py -el data/edge_list.tsv -gs data/gene_scores.tsv -o results/example_output.tsv

for n in range(4400):
    assert network[n] in score[n]
